In [1]:
import gunpowder as gp
from funlib.persistence import open_ds #may need to change to ome later

/home/S-EK/conda/envs/bs/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# declare arrays to use in the pipeline
raw = gp.ArrayKey('RAW')
labels = gp.ArrayKey('LABELS')
mask = gp.ArrayKey('MASK')
gt_affs = gp.ArrayKey('GT_AFFS')
gt_affs_mask = gp.ArrayKey('GT_AFFS_MASK')

In [3]:
raw_array = open_ds("/mnt/efs/aimbl_2025/student_data/S-EK/EK_transfer/GT_movie1/crop_1.zarr/raw")
labels_array = open_ds("/mnt/efs/aimbl_2025/student_data/S-EK/EK_transfer/GT_movie1/crop_1.zarr/labels")
mask_array = open_ds("/mnt/efs/aimbl_2025/student_data/S-EK/EK_transfer/GT_movie1/crop_1.zarr/mask")

In [4]:
print(raw_array.voxel_size)
print(labels_array.voxel_size)
print(mask_array.voxel_size)

(1000, 170, 170)
(1000, 170, 170)
(1000, 170, 170)


In [5]:
# create "pipeline" consisting only of a data source
source_raw = gp.ArraySource( key= raw, array= raw_array, interpolatable= True)
source_labels = gp.ArraySource( key= labels, array= labels_array, interpolatable= False)
source_mask = gp.ArraySource( key= mask, array= mask_array, interpolatable= False)

In [6]:
random_location = gp.RandomLocation()

In [ ]:
train.gp.torch.Train()

In [7]:
snapshot = gp.Snapshot(
    dataset_names={raw:"raw", labels:"labels", mask:"mask", gt_affs: "affs", gt_affs_mask: "affS_mask"},
    dataset_dtypes={gt_affs: 'float32'}
)

In [8]:
normalization = gp.Normalize(array= raw)

In [9]:
deform = gp.DeformAugment(
    gp.Coordinate((5100, 5100)),
    (340,340),
    spatial_dims=2,
    graph_raster_voxel_size=raw_array.voxel_size[1:]
)

In [10]:
neighborhood = [
    (1, 0 ,0),
    (0, 1, 0),
    (0, 0, 1),
    (2, 0, 0),
    (0, 5, 0),
    (0, 0, 5)
]
add_affs = gp.AddAffinities(
    affinity_neighborhood=neighborhood,
    labels=labels,
    affinities=gt_affs,
    unlabelled=mask,
    affinities_mask=gt_affs_mask
)

In [11]:
pipeline = (source_raw,source_labels,source_mask) + gp.MergeProvider() + random_location + normalization + gp.SimpleAugment(transpose_only=[1,2]) + deform + add_affs + snapshot 

In [ ]:
test_shape = raw_array.shape

(16, 1492, 1492)

In [ ]:
# formulate a request for "raw"

input_size = gp.Coordinate((7, 512, 512)) * raw_array.voxel_size
output_size = gp.Coordinate((3, 256, 256)) * raw_array.voxel_size

request = gp.BatchRequest()

request.add(raw, input_size)
request.add(labels, output_size)
request.add(mask, output_size)
request.add(gt_affs, output_size)
request.add(gt_affs_mask, output_size)

# request[raw] = gp.Roi((5000, 119000, 119000), (10000, 85000, 85000)) #always in world units
# request[labels] = gp.Roi((5000, 119000, 119000), (10000, 85000, 85000)) #always in world units
# request[mask] = gp.Roi((5000, 119000, 119000), (10000, 85000, 85000)) #always in world units
# request[gt_affs] = gp.Roi((5000, 119000, 119000), (10000, 85000, 85000)) #always in world units
# request[gt_affs_mask] = gp.Roi((5000, 119000, 119000), (10000, 85000, 85000)) #always in world units

In [ ]:
# build the pipeline...
with gp.build(pipeline):
  for i in range(1):
    # ...and request a batch
    print(i)
    batch = pipeline.request_batch(request)


In [6]:
from ..src import train

ImportError: attempted relative import with no known parent package

In [ ]:
x = {'input':raw}

In [ ]:
raw_gp_array = x['input']
torch_raw = 